In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from tqdm import tqdm
import copy

In [2]:
# pip install bert-tensorflow
# Do not install this

In [3]:
# pip install bert-for-tf2
# already installed

In [4]:
# pip install sentencepiece
# already installed

In [5]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import gc
import os
import numpy as np
import collections
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import ModelCheckpoint
import random

In [7]:
# generator to split list into chunks
def chunks(lst, n):
    if not n:
        raise Exception("n must be a positive integer")

    for i in range(0, len(lst), n):
        yield lst[i:i + n]


In [8]:
def transliterate(line):
    cedilla2latin = [[u'Á', u'A'], [u'á', u'a'], [u'Č', u'C'], [u'č', u'c'], [u'Š', u'S'], [u'š', u's']]
    tr = dict([(a[0], a[1]) for (a) in cedilla2latin])
    new_line = ""
    for letter in line:
        if letter in tr:
            new_line += tr[letter]
        else:
            new_line += letter
    return new_line

In [9]:
def text_cleaner(text,
                 deep_clean=True,
                 stem= True,
                 stop_words=True,
                 translite_rate=True):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning

    ]

    if deep_clean:
        text = text.replace(".", "")
        text = text.replace("[", " ")
        text = text.replace(",", " ")
        text = text.replace("]", " ")
        text = text.replace("(", " ")
        text = text.replace(")", " ")
        text = text.replace("\"", "")
        text = text.replace("-", " ")
        text = text.replace("=", " ")
        text = text.replace("?", " ")
        text = text.replace("!", " ")

        for rule in rules:
            for (k, v) in rule.items():
                regex = re.compile(k)
                text = regex.sub(v, text)
            text = text.rstrip()
            text = text.strip()
        text = text.replace('+', ' ').replace('.', ' ').replace(',', ' ').replace(':', ' ')
        text = re.sub("(^|\W)\d+($|\W)", " ", text)
        if translite_rate:
            text = transliterate(text)
        if stem:
            text = PorterStemmer().stem(text)
        text = WordNetLemmatizer().lemmatize(text)
        if stop_words:
            stop_words = set(stopwords.words('english'))
            word_tokens = word_tokenize(text)
            text = [w for w in word_tokens if not w in stop_words]
            text = ' '.join(str(e) for e in text)
    else:
        for rule in rules:
            for (k, v) in rule.items():
                regex = re.compile(k)
                text = regex.sub(v, text)
            text = text.rstrip()
            text = text.strip()
    return text.lower()

In [10]:
#make pre-train for bert
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [11]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from collections import Counter
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download("stopwords")
cachedStopWords = stopwords.words("english")
from sklearn.model_selection import train_test_split


#Function to convert html body of data to text
def convert_html_to_text(data):
    soup = bs(data,'html.parser')
    body = soup.get_text()
    return body

#Prepare data
mytrain = pd.read_csv("../freecode_data.csv")

#print(mytrain['Tags'])

mytrain['Title'] = mytrain['title'].apply(text_cleaner)

mytrain['Body'] = mytrain['question'].apply(text_cleaner)

mytrain['Text'] = mytrain['Title'] + "\n" + mytrain['Body']




#Frequency of each tag

vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
tag_bow = vectorizer.fit_transform(mytrain['tag'])

tags = vectorizer.get_feature_names()

freq = tag_bow.sum(axis=0).A1
tag_to_count_map = dict(zip(tags, freq))

lst = []
for key, value in tag_to_count_map.items():
  lst.append([key, value]) 


tag_df = pd.DataFrame(lst, columns=['tag', 'Counts'])
print(tag_df.head())

tag_df_sorted = tag_df.sort_values(['Counts'], ascending=False)


print("{} tags are used more than 80 times".format(tag_df_sorted[tag_df_sorted["Counts"]>50]))



most_common_tag = tag_df_sorted[tag_df_sorted["Counts"]>80].shape[0]

print("most_common_tag:",most_common_tag)


X = mytrain['Text'].tolist()
#print(X[7])


#prepare tags
list_of_tags = []
for item in mytrain['tag']:
    temp = item.split(" ")
    for word in temp:
        if not word in list_of_tags:
            list_of_tags.append(word)
            

list_of_all_tags = []
for item in mytrain['tag']:
    temp = item.split(" ")
    for word in temp:
        list_of_all_tags.append(word)


counts = Counter(list_of_all_tags)

most_occurs = [(k, v) for k, v in counts.items() if v >= 80] # todo: check this with Navid
print("most_occurs", most_occurs)

tags = []
for item in most_occurs:
    tags.append(item[0])
    #print("tags:",tags)



#print("tags:",tags)
y = []
S=0
for item in mytrain['tag']:
    self_tags = []
    itemsplitted = item.split(" ")
    
    for word in tags:
        if word in itemsplitted:
            self_tags.append(1)
        else:
            self_tags.append(0)
    
    values = np.array(self_tags)
    
    Y=all(values == 0)
    if Y==True:
        
        del X[S]
        S=S-1
    else:
        y.append(np.array(self_tags))
    S=S+1    
    

y_list = []
for elem in y:
    y_list.append(elem.tolist())
    
def tokenize_data(data):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data))



tokenized_data = [tokenize_data(data) for data in X]


tokenized_data_train,tokenized_data_test,y_list_train, y_list_test = train_test_split(tokenized_data, y_list, test_size = 10000,random_state = 42)
print("Number of data points in training data :", len(tokenized_data_train))
print("Number of data points in test data :", len(tokenized_data_test))

for c,item in enumerate(y_list_test):
    y_list_test[c] = np.array(item)

[nltk_data] Downloading package stopwords to /home/elias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/elias/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


                                 tag  Counts
0         (devexpress-codegenerator)       1
1                               *bsd       1
2      .jrxml-specifications-support       1
3                               .net      29
4  .net-client-profile-compatibility       1
                       tag  Counts
7395  software-development    9655
3790              internet    8171
8594                   web    6316
4277             libraries    5758
5111            multimedia    4833
...                    ...     ...
304              animation      52
8508                  voip      51
5738         page-counters      51
528         authentication      51
5777                parser      51

[273 rows x 2 columns] tags are used more than 80 times
most_common_tag: 220
most_occurs [('Systems-Administration', 1844), ('Information-Management', 1349), ('Issue-Tracking', 197), ('PHP', 491), ('php-classes', 824), ('Security', 1978), ('LDAP', 92), ('SQL', 85), ('Operating-Systems', 492), ('Monitoring', 

In [12]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [13]:
#Function to show progress in consule :)
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [14]:
#bert text model
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")

                                    
            

        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 


        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [36]:
import sys
import os
import nltk
from nltk.corpus import reuters
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import random
import math
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import hamming_loss
import statistics 


whole_predictions = []
whole_real_predictions = []
whole_threshold_predictions = []

one=0

#predict for each label individualy

for i in range(len(y_list_train[0])):
    print("\n" + str(i)+"\'th label prediction started")
    count_zero=0
    count_one=0
    new_label=[]
    new_tokenized_data_train=[]
    label = column(y_list_train,i)
    count_one=sum(label)
    print("count_one",count_one)
    
    for k in range(len(label)):
        if count_zero< count_one and label[k]==0:
            new_label.append(0)
            new_tokenized_data_train.append(tokenized_data_train[k])
            count_zero=count_zero+1
        if label[k]==1:
            new_label.append(1)
            new_tokenized_data_train.append(tokenized_data_train[k])

            
    print("count_zero",count_zero)        
    data_with_len = [[data,new_label[j],len(data)]
                     for j,data in enumerate(new_tokenized_data_train)]

    data_with_len.sort(key=lambda x: x[2])
    sorted_data_labels = [(data_lab[0], data_lab[1]) for data_lab in data_with_len]
    processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_data_labels, output_types=(tf.int32, tf.int32))
    BATCH_SIZE = 32
    batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
    TOTAL_BATCHES = math.ceil(len(sorted_data_labels) / BATCH_SIZE)
#     TEST_BATCHES = TOTAL_BATCHES // TOTAL_BATCHES
#     batched_dataset.shuffle(TOTAL_BATCHES)
#     test_data = batched_dataset.take(TEST_BATCHES)
#     train_data = batched_dataset.skip(TEST_BATCHES)
    train_data = batched_dataset
    
   
    VOCAB_LENGTH = len(tokenizer.vocab)
    EMB_DIM = 260
    CNN_FILTERS = 50
    DNN_UNITS = 256
    OUTPUT_CLASSES = 2

    DROPOUT_RATE = 0.2

    NB_EPOCHS = 6

    text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

    if OUTPUT_CLASSES == 2:
        text_model.compile(loss="binary_crossentropy",
                           optimizer="adam",
                           metrics=["acc"])
    else:
        text_model.compile(loss="sparse_categorical_crossentropy",
                           optimizer="adam",
                           metrics=["sparse_categorical_acc"])

    text_model.fit(train_data, epochs=NB_EPOCHS)

    self_label_predictions = []
    self_threshold_predictions = []
    self_label_real_values = []
    print("Predicting " + str(i) + "th label...")
    
    for chunk in tqdm(list(chunks(tokenized_data_test, BATCH_SIZE))):
#         if e%2==0:
#             progress(e,len(tokenized_data_test))
#         print("type(chunk[0]): {0}".format(type(chunk[0])))
    
        processed_chunk_dataset = tf.data.Dataset.from_generator(lambda: chunk, output_types=(tf.int32))
        batched_chunk_dataset = processed_chunk_dataset.padded_batch(BATCH_SIZE, padded_shapes=(None,))
    
        rows = text_model.predict(batched_chunk_dataset, batch_size=len(chunk))
        for res in rows:
            self_label_real_values.append(res[0])

            if res[0] > 0.93:
                self_threshold_predictions.append(res[0])
            else :
                self_threshold_predictions.append(0.0)

    whole_threshold_predictions.append(self_threshold_predictions)
    whole_real_predictions.append(self_label_real_values)



0'th label prediction started
count_one 1424
count_zero 1424
Epoch 1/6
89/89 [==============================] - 1s 11ms/step - loss: 0.6518 - acc: 0.6545
Epoch 2/6
89/89 [==============================] - 1s 11ms/step - loss: 0.3041 - acc: 0.8715
Epoch 3/6
89/89 [==============================] - 1s 11ms/step - loss: 0.0689 - acc: 0.9817
Epoch 4/6
89/89 [==============================] - 1s 11ms/step - loss: 0.0123 - acc: 0.9979
Epoch 5/6
89/89 [==============================] - 1s 11ms/step - loss: 0.0040 - acc: 0.9996
Epoch 6/6
89/89 [==============================] - 1s 11ms/step - loss: 0.0016 - acc: 1.0000
Predicting 0th label...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 15.88it/s]



1'th label prediction started
count_one 1023
count_zero 1023
Epoch 1/6
64/64 [==============================] - 1s 11ms/step - loss: 0.6628 - acc: 0.6183
Epoch 2/6
64/64 [==============================] - 1s 11ms/step - loss: 0.4102 - acc: 0.8226
Epoch 3/6
64/64 [==============================] - 1s 11ms/step - loss: 0.0946 - acc: 0.9765
Epoch 4/6
64/64 [==============================] - 1s 10ms/step - loss: 0.0170 - acc: 0.9990
Epoch 5/6
64/64 [==============================] - 1s 11ms/step - loss: 0.0048 - acc: 1.0000
Epoch 6/6
64/64 [==============================] - 1s 11ms/step - loss: 0.0016 - acc: 1.0000
Predicting 1th label...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 15.88it/s]



2'th label prediction started
count_one 144
count_zero 144
Epoch 1/6
9/9 [==============================] - 1s 13ms/step - loss: 0.6961 - acc: 0.5243
Epoch 2/6
9/9 [==============================] - 0s 10ms/step - loss: 0.5955 - acc: 0.9618
Epoch 3/6
9/9 [==============================] - 0s 11ms/step - loss: 0.4417 - acc: 0.9965
Epoch 4/6
9/9 [==============================] - 0s 11ms/step - loss: 0.2289 - acc: 1.0000
Epoch 5/6
9/9 [==============================] - 0s 11ms/step - loss: 0.0764 - acc: 1.0000
Epoch 6/6
9/9 [==============================] - 0s 11ms/step - loss: 0.0188 - acc: 1.0000
Predicting 2th label...


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.17it/s]



219'th label prediction started
count_one 63
count_zero 63
Epoch 1/6
4/4 [==============================] - 1s 14ms/step - loss: 0.7108 - acc: 0.4286
Epoch 2/6
4/4 [==============================] - 0s 11ms/step - loss: 0.6364 - acc: 0.5317
Epoch 3/6
4/4 [==============================] - 0s 9ms/step - loss: 0.5619 - acc: 1.0000
Epoch 4/6
4/4 [==============================] - 0s 11ms/step - loss: 0.4646 - acc: 1.0000
Epoch 5/6
4/4 [==============================] - 0s 12ms/step - loss: 0.3434 - acc: 1.0000
Epoch 6/6
4/4 [==============================] - 0s 11ms/step - loss: 0.2273 - acc: 1.0000
Predicting 219th label...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:20<00:00, 15.09it/s]



220'th label prediction started
count_one 62
count_zero 62
Epoch 1/6
4/4 [==============================] - 1s 11ms/step - loss: 0.6942 - acc: 0.5484
Epoch 2/6
4/4 [==============================] - 0s 12ms/step - loss: 0.6318 - acc: 1.0000
Epoch 3/6
4/4 [==============================] - 0s 11ms/step - loss: 0.5544 - acc: 1.0000
Epoch 4/6
4/4 [==============================] - 0s 11ms/step - loss: 0.4430 - acc: 1.0000
Epoch 5/6
4/4 [==============================] - 0s 10ms/step - loss: 0.3131 - acc: 1.0000
Epoch 6/6
4/4 [==============================] - 0s 10ms/step - loss: 0.1859 - acc: 1.0000
Predicting 220th label...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:20<00:00, 15.42it/s]


In [ ]:
whole_threshold_predictions = list(map(list, zip(*whole_threshold_predictions)))
whole_real_predictions = list(map(list, zip(*whole_real_predictions)))




        
#@k Formulas :
K_list= [3,5,10]
for U in range(len(K_list)):
    K_tag_y_list_test = []
    k = K_list[U]
    sigma_recalls = 0
    sigma_precisions = 0
    sigma_f1score = 0

    K_tag_y_list_test = y_list_test
    for f in tqdm(range(len(K_tag_y_list_test))):
#         progress(f,len(K_tag_y_list_test))
        currentitem = np.array(whole_threshold_predictions[f])

        top_k_indexes = (-currentitem).argsort()[:k]
        
        for C in top_k_indexes:
            if whole_threshold_predictions[f][C] == 0.0 :
                top_k_indexes = top_k_indexes[top_k_indexes != C]
      
        intercep = 0
        for numb in top_k_indexes:
            if K_tag_y_list_test[f][numb] == 1 :
                intercep += 1
        num_of_exists_tags = np.count_nonzero(K_tag_y_list_test[f] == 1)

        if len(top_k_indexes) == 0 :
            self_recall_k=0
        elif len(top_k_indexes) >= num_of_exists_tags :
            self_recall_k = intercep / num_of_exists_tags
        elif len(top_k_indexes) < num_of_exists_tags :
            self_recall_k = intercep / len(top_k_indexes)
        if len(top_k_indexes)==0:
             self_precisions_k=0 
        else:    
            self_precisions_k = intercep / len(top_k_indexes)
        if self_precisions_k==0 and self_recall_k==0:
            self_f1_score_k=0
        else:    
            self_f1_score_k = 2 * ((self_precisions_k*self_recall_k)/(self_precisions_k+self_recall_k))
        sigma_recalls += self_recall_k
        sigma_precisions += self_precisions_k
        sigma_f1score += self_f1_score_k

    recall_k = sigma_recalls / len(K_tag_y_list_test)
    precisions_k = sigma_precisions / len(K_tag_y_list_test)
    f1score_k = sigma_f1score / len(K_tag_y_list_test)
    print("\n")
    print("Recall@"+ str(K_list[U])+" = " + str(recall_k))
    print("Precision@"+ str(K_list[U])+" = " + str(precisions_k))
    print("f1score@"+ str(K_list[U])+" = " + str(f1score_k))

In [27]:
text_model.summary()

Model: "text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  7935720   
_________________________________________________________________
conv1d_18 (Conv1D)           multiple                  26050     
_________________________________________________________________
conv1d_19 (Conv1D)           multiple                  39050     
_________________________________________________________________
conv1d_20 (Conv1D)           multiple                  52050     
_________________________________________________________________
global_max_pooling1d_6 (Glob multiple                  0         
_________________________________________________________________
dense_12 (Dense)             multiple                  38656     
_________________________________________________________________
dropout_6 (Dropout)          multiple                  0